In [1]:
import pandas as pd
import numpy as np
import os

In [20]:
base = pd.read_csv('./feature_csv/base_new.csv',index_col=0)

#resident_pop_data = pd.read_csv('01_제공데이터/03_거주인구_RESIDENT_POP.CSV',low_memory=False,encoding='CP949')

# 카드 사용 내역

## 데이터 불러오기

In [2]:
base = pd.read_csv('./feature_csv/base_new.csv',index_col=0)
base.drop(columns=['pay_amt'],inplace=True)
card = pd.read_csv('01_제공데이터/04_음식관련 카드소비_CARD_SPENDING.CSV',encoding='cp949')

In [3]:
# 날짜 타입 변경
card['year'] = pd.to_datetime(card['base_date']).dt.year
card['month'] = pd.to_datetime(card['base_date']).dt.month
card['date'] = pd.to_datetime(card['base_date']).dt.day

In [4]:
base

,year,month,date,emd_nm,em_cnt,em_g
0,2018,1,1,건입동,668,1708250
1,2018,1,1,남원읍,304,1239600
2,2018,1,1,노형동,3903,9357900
3,2018,1,1,대륜동,650,1717700
4,2018,1,1,대정읍,309,1264950
...,...,...,...,...,...,...
50518,2021,6,30,한경면,269,973850
50519,2021,6,30,한림읍,870,3298100
50520,2021,6,30,화북동,2255,3749600
50521,2021,6,30,효돈동,313,1011050


## 데이터 가공

In [5]:
# 행정동 별 카테고리 당 카드 사용건수/사용 금액 총합한 데이터 프레임을 각각 생성
cat_cnt = card.groupby(['year','month','date','emd_nm','mct_cat_nm'])['use_cnt'].sum().reset_index()
cat_amt = card.groupby(['year','month','date','emd_nm','mct_cat_nm'])['use_amt'].sum().reset_index()

# 모든 행정동을 총합한 "전체" 항목을 각 데이터프레임의 행정동 column(emd_nm)에 추가 => 
temp = card.groupby(['year','month','date','mct_cat_nm'])['use_cnt'].sum().reset_index()
temp['emd_nm'] = '전체'
temp = temp[['year','month','date','emd_nm','mct_cat_nm','use_cnt']]
cat_cnt = pd.concat([cat_cnt,temp])

temp = card.groupby(['year','month','date','mct_cat_nm'])['use_amt'].sum().reset_index()
temp['emd_nm'] = '전체'
temp = temp[['year','month','date','emd_nm','mct_cat_nm','use_amt']]
cat_amt = pd.concat([cat_amt,temp])

In [6]:
# 행정동 별 모든 카테고리의 카드 사용건수/사용금액 총합
total_cnt = card.groupby(['year','month','date','emd_nm'])['use_cnt'].sum().reset_index()
total_amt = card.groupby(['year','month','date','emd_nm'])['use_amt'].sum().reset_index()

# 모든 행정동을 총합한 "전체" 항목을 각 데이터프레임의 행정동 column(emd_nm)에 추가
temp = card.groupby(['year','month','date'])['use_cnt'].sum().reset_index()
temp['emd_nm'] = '전체'
temp = temp[['year','month','date','emd_nm','use_cnt']]
total_cnt = pd.concat([total_cnt,temp])

temp = card.groupby(['year','month','date'])['use_amt'].sum().reset_index()
temp['emd_nm'] = '전체'
temp = temp[['year','month','date','emd_nm','use_amt']]
total_amt = pd.concat([total_amt,temp])

In [7]:
# 배달과 슈퍼마켓 카테고리만을 추출 후 모든 카테고리의 총합과 left outter join
# => 사용량에 대한 df, 사용건수에 대한 df 두 개의 데이터프레임 생성
cat_cnt = cat_cnt[(cat_cnt['mct_cat_nm'] == "배달") | (cat_cnt['mct_cat_nm'] == "마트/슈퍼마켓")].reset_index(drop=True)
cat_cnt.columns = ['year','month','date','emd_nm','mct_cat_nm','cat_use_cnt']
merge_cat_cnt = pd.merge(cat_cnt, total_cnt,how='left',on=['year','month','date','emd_nm'])

cat_amt = cat_amt[(cat_amt['mct_cat_nm'] == "배달") | (cat_amt['mct_cat_nm'] == "마트/슈퍼마켓")].reset_index(drop=True)
cat_amt.columns = ['year','month','date','emd_nm','mct_cat_nm','cat_use_amt']
merge_cat_amt = pd.merge(cat_amt, total_amt,how='left',on=['year','month','date','emd_nm'])

In [8]:
merge_cat_cnt

,year,month,date,emd_nm,mct_cat_nm,cat_use_cnt,use_cnt
0,2018,1,1,건입동,마트/슈퍼마켓,589,1806
1,2018,1,1,건입동,배달,5,1806
2,2018,1,1,구좌읍,마트/슈퍼마켓,329,1180
3,2018,1,1,구좌읍,배달,5,1180
4,2018,1,1,남원읍,마트/슈퍼마켓,466,1581
...,...,...,...,...,...,...,...
110205,2021,6,28,전체,배달,13873,138451
110206,2021,6,29,전체,마트/슈퍼마켓,42589,142605
110207,2021,6,29,전체,배달,14642,142605
110208,2021,6,30,전체,마트/슈퍼마켓,41468,143124


In [9]:
merge_cat_amt

,year,month,date,emd_nm,mct_cat_nm,cat_use_amt,use_amt
0,2018,1,1,건입동,마트/슈퍼마켓,19891147,61606900
1,2018,1,1,건입동,배달,54010,61606900
2,2018,1,1,구좌읍,마트/슈퍼마켓,11489793,43303254
3,2018,1,1,구좌읍,배달,66285,43303254
4,2018,1,1,남원읍,마트/슈퍼마켓,24828201,69402997
...,...,...,...,...,...,...,...
110205,2021,6,28,전체,배달,315628374,4291345343
110206,2021,6,29,전체,마트/슈퍼마켓,1400197354,4412362166
110207,2021,6,29,전체,배달,334820931,4412362166
110208,2021,6,30,전체,마트/슈퍼마켓,1457149974,4788734079


In [10]:
# 사용량 : 카테고리에 대해 long form인 데이터를 wide form으로 피벗 테이블 생성 후 전체 카드 사용량 데이터와 join
deli = merge_cat_amt[merge_cat_amt['mct_cat_nm'] == '배달'][['year','month','date','emd_nm','cat_use_amt']].reset_index(drop=True)
market = merge_cat_amt[merge_cat_amt['mct_cat_nm'] == '마트/슈퍼마켓'][['year','month','date','emd_nm','cat_use_amt']].reset_index(drop=True)

deli.columns = ['year','month','date','emd_nm','delivery_amt']
market.columns = ['year','month','date','emd_nm','market_amt']

total_amt.columns = ['year','month','date','emd_nm','card_use_amt']

df1 = pd.merge(deli, market, on=['year','month','date','emd_nm'])
df1 = pd.merge(total_amt, df1, on=['year','month','date','emd_nm'])

In [11]:
# 사용건수 : 카테고리에 대해 long form인 데이터를 wide form으로 피벗 테이블 생성 후 전체 카드 사용횟수 데이터와 join
deli = merge_cat_cnt[merge_cat_cnt['mct_cat_nm'] == '배달'][['year','month','date','emd_nm','cat_use_cnt']].reset_index(drop=True)
market = merge_cat_cnt[merge_cat_cnt['mct_cat_nm'] == '마트/슈퍼마켓'][['year','month','date','emd_nm','cat_use_cnt']].reset_index(drop=True)

deli.columns = ['year','month','date','emd_nm','delivery_cnt']
market.columns = ['year','month','date','emd_nm','market_cnt']

total_cnt.columns = ['year','month','date','emd_nm','card_use_cnt']

df2 = pd.merge(deli, market, on=['year','month','date','emd_nm'])
df2 = pd.merge(total_cnt, df2, on=['year','month','date','emd_nm'])

In [12]:
df1 # 카드 사용량에 대한 데이터 프레임

,year,month,date,emd_nm,card_use_amt,delivery_amt,market_amt
0,2018,1,1,건입동,61606900,54010,19891147
1,2018,1,1,구좌읍,43303254,66285,11489793
2,2018,1,1,남원읍,69402997,644192,24828201
3,2018,1,1,노형동,403571632,2185932,168640726
4,2018,1,1,대륜동,72541912,396728,37971731
...,...,...,...,...,...,...,...
52743,2021,6,26,전체,4792478704,506894912,1085600438
52744,2021,6,27,전체,5046833648,519754007,1870966940
52745,2021,6,28,전체,4291345343,315628374,1458525569
52746,2021,6,29,전체,4412362166,334820931,1400197354


In [13]:
df2 # 카드 사용건수에 대한 데이터 프레임

,year,month,date,emd_nm,card_use_cnt,delivery_cnt,market_cnt
0,2018,1,1,건입동,1806,5,589
1,2018,1,1,구좌읍,1180,5,329
2,2018,1,1,남원읍,1581,25,466
3,2018,1,1,노형동,12507,123,4905
4,2018,1,1,대륜동,2032,20,859
...,...,...,...,...,...,...,...
52743,2021,6,26,전체,159384,21348,36436
52744,2021,6,27,전체,156986,21900,47729
52745,2021,6,28,전체,138451,13873,42002
52746,2021,6,29,전체,142605,14642,42589


## 결측값 처리
특정 행정동의 경우 카드 소비를 안 한 날도 확인되었기 때문에, interploate를 사용하여 결측값을 채움<br>
* **interploate** : 특정의 동일 값으로 채우는 방식(filling, imputation) 보다는, 실측값과 실측값 사이의 결측값을 마치 '그라데이션(gradation)' 기법으로 색깔을 조금씩 변화시켜가면서 부드럽게 채워나가는 보간법

In [14]:
# 연, 월, 일, 행정동을 기준으로 base와 df1, df2를 모두 병합
t = pd.merge(base,df1,on=['year','month','date','emd_nm'],how='left')
t = pd.merge(t,df2,on=['year','month','date','emd_nm'],how='left')

null_emd = t[t['card_use_cnt'].isnull()==True].emd_nm.unique() # 결측값이 하나라도 있는 행정동 추출 후, 행정동 별로 결측치 처리

lst = []
for emd in null_emd:
    # 1. 기존 인덱스를 제거하고 연-월-일의 날짜변수를 인덱스로 삽입
    temp = t[t['emd_nm'] == emd]
    t.drop(temp.index,inplace=True)
    temp['base_date'] = pd.to_datetime(temp['year'].astype(str)+'-'+temp['month'].astype(str)+'-'+temp['date'].astype(str))
    temp = temp.set_index('base_date')
    
    # 2. 각 열에 대해서 결측치 보간
    temp['card_use_cnt'] = temp['card_use_cnt'].interpolate(method='time')
    temp['delivery_cnt'] = temp['delivery_cnt'].interpolate(method='time')
    temp['market_cnt'] = temp['market_cnt'].interpolate(method='time')
    
    temp['card_use_amt'] = temp['card_use_amt'].interpolate(method='time')
    temp['delivery_amt'] = temp['delivery_amt'].interpolate(method='time')
    temp['market_amt'] = temp['market_amt'].interpolate(method='time')
    
    # 3. 가장 첫 번째 데이터가 결측치인 경우 interploate 보간법이 적용되지 않으므로, 중앙값으로 값 대체
    temp = temp.fillna(temp.median())
    temp = temp.reset_index(drop=True)
    lst.append(temp)

<ipython-input-14-f7c3b573b732>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['base_date'] = pd.to_datetime(temp['year'].astype(str)+'-'+temp['month'].astype(str)+'-'+temp['date'].astype(str))


In [15]:
pd.concat([t,pd.concat(lst)]).to_csv('feature_csv/card_feature.csv')

,year,month,date,emd_nm,em_cnt,em_g,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt
2,2018,1,1,노형동,3903,9357900,403571632.0,2185932.0,168640726.0,12507.0,123.0,4905.0
11,2018,1,1,삼양동,1580,3997300,156972711.0,768415.0,78558822.0,4060.0,34.0,1664.0
15,2018,1,1,아라동,1881,4372000,189213430.0,792965.0,75143475.0,5608.0,39.0,1920.0
18,2018,1,1,연동,3091,7264350,357638950.0,3761060.0,153734260.0,10110.0,182.0,3903.0
26,2018,1,1,이도2동,3536,9033700,359636524.0,3585149.0,139494229.0,10783.0,187.0,3643.0
...,...,...,...,...,...,...,...,...,...,...,...,...
595,2021,6,26,구좌읍,503,1426300,84378688.0,2848977.0,20621944.0,2108.0,141.0,493.0
596,2021,6,27,구좌읍,565,1382150,60689853.0,2296733.0,14857916.0,1825.0,121.0,402.0
597,2021,6,28,구좌읍,487,1263300,88674022.0,2279596.0,24662543.0,1855.0,91.0,458.0
598,2021,6,29,구좌읍,479,1336600,87047214.0,3037461.0,22597074.0,2048.0,121.0,503.0


----------------------------------

In [21]:
base['base_date'] = base['year'].astype('str')+'-'+base['month'].astype('str')+'-'+base['date'].astype('str')
base['base_date'] = pd.to_datetime(base['base_date'])
base.drop(columns=['year','month','date'],inplace=True)
base = base[['base_date','emd_nm','em_cnt','em_g']]
base

,base_date,emd_nm,em_cnt,em_g
0,2018-01-01,건입동,668,1708250
1,2018-01-01,남원읍,304,1239600
2,2018-01-01,노형동,3903,9357900
3,2018-01-01,대륜동,650,1717700
4,2018-01-01,대정읍,309,1264950
...,...,...,...,...
50518,2021-06-30,한경면,269,973850
50519,2021-06-30,한림읍,870,3298100
50520,2021-06-30,화북동,2255,3749600
50521,2021-06-30,효돈동,313,1011050


In [22]:
# 카드 이용 데이터 merge
data = pd.merge(base,pd.concat([t,pd.concat(lst)]))#.to_csv('feature_csv/merge_210913.csv',index=False)
data['year']=data['year'].astype(str)
data['month']=data['month'].astype(str)
data['date']=data['date'].astype(str)

data['base_date']=data['year']+"-"+data['month']+"-"+data['date']

data['base_date']=pd.to_datetime(data['base_date'])

data.drop(['year','month','date'],axis=1,inplace=True)

In [24]:
db = pd.read_csv('feature_csv/youth_frgn_pop.csv',index_col=0)
db['base_date'] = pd.to_datetime(db['base_date'])

In [25]:
tmp = pd.merge(db,data,on=['base_date','emd_nm','em_g'])

In [27]:
data = pd.read_csv('feature_csv/bolim.csv',index_col=0)

data['year']=data['year'].astype(str)
data['month']=data['month'].astype(str)
data['date']=data['date'].astype(str)

data['base_date']=data['year']+"-"+data['month']+"-"+data['date']

data['base_date']=pd.to_datetime(data['base_date'])

data.drop(['year','month','date'],axis=1,inplace=True)

In [30]:
pd.read_csv('feature_csv/final_final_train_0913.csv',index_col=0)

,base_date,emd_nm,em_g,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop
0,2018-01-01,전체,85755000,679922,535888.570169,3.786578e+09,20891720.0,1.602637e+09,108983.0,1057.0,40848.0,32512,7.210523e+06,8.301891e+05,54466.952499,2215.223209
1,2018-01-02,전체,90217650,679922,534509.814556,3.406059e+09,15597155.0,1.286966e+09,107164.0,839.0,38738.0,34664,6.761153e+06,1.368438e+06,53973.690842,2221.350506
2,2018-01-03,전체,80277600,679922,532472.061474,3.499232e+09,13453350.0,1.292679e+09,109765.0,728.0,38284.0,31025,6.770127e+06,1.369782e+06,53705.165769,2172.085577
3,2018-01-04,전체,80138700,679922,514678.201983,3.329829e+09,14592029.0,1.215080e+09,107110.0,740.0,36878.0,31540,6.993326e+06,1.351504e+06,53151.118467,2111.044522
4,2018-01-05,전체,75798600,679922,527383.565802,3.733755e+09,21105231.0,1.290936e+09,112064.0,1009.0,37686.0,29709,6.825653e+06,1.317354e+06,53360.869746,2055.834760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50881,2021-06-30,표선면,1377450,12681,10137.380983,9.063287e+07,1855783.0,3.831039e+07,1336.0,75.0,423.0,370,1.614560e+05,2.405863e+04,914.664667,12.910869
50882,2021-06-30,한경면,973850,9495,7257.895822,4.526361e+07,697661.0,1.767975e+07,1286.0,40.0,423.0,269,1.041218e+05,1.633182e+04,608.427552,22.165953
50883,2021-06-30,한림읍,3298100,24462,20632.727912,1.301568e+08,5508665.0,4.207137e+07,3044.0,257.0,669.0,870,3.018026e+05,5.060204e+04,2064.041876,48.329788
50884,2021-06-30,화북동,3749600,24630,24117.268821,1.918163e+08,13370520.0,4.846083e+07,5644.0,644.0,1645.0,2255,2.433203e+05,5.097469e+04,1810.431884,18.483512


In [34]:
pd.merge(tmp,data,on=['base_date','emd_nm','em_g']).drop(columns = ['em_cnt_x'].to_csv('feature_csv/final_final_train_0913.csv')

,base_date,emd_nm,em_g,youth_pop,total_frgn_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt_y,pay_amt,visit_pop,work_pop
0,2018-01-01,전체,85755000,54466.952499,2215.223209,3.786578e+09,20891720.0,1.602637e+09,108983.0,1057.0,40848.0,32512,2550543,7.210523e+06,8.301891e+05
1,2018-01-02,전체,90217650,53973.690842,2221.350506,3.406059e+09,15597155.0,1.286966e+09,107164.0,839.0,38738.0,34664,2681290,6.761153e+06,1.368438e+06
2,2018-01-03,전체,80277600,53705.165769,2172.085577,3.499232e+09,13453350.0,1.292679e+09,109765.0,728.0,38284.0,31025,2387761,6.770127e+06,1.369782e+06
3,2018-01-04,전체,80138700,53151.118467,2111.044522,3.329829e+09,14592029.0,1.215080e+09,107110.0,740.0,36878.0,31540,2382283,6.993326e+06,1.351504e+06
4,2018-01-05,전체,75798600,53360.869746,2055.834760,3.733755e+09,21105231.0,1.290936e+09,112064.0,1009.0,37686.0,29709,2253564,6.825653e+06,1.317354e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50917,2021-06-30,표선면,1377450,914.664667,12.910869,9.063287e+07,1855783.0,3.831039e+07,1336.0,75.0,423.0,370,41235,1.614560e+05,2.405863e+04
50918,2021-06-30,한경면,973850,608.427552,22.165953,4.526361e+07,697661.0,1.767975e+07,1286.0,40.0,423.0,269,29152,1.041218e+05,1.633182e+04
50919,2021-06-30,한림읍,3298100,2064.041876,48.329788,1.301568e+08,5508665.0,4.207137e+07,3044.0,257.0,669.0,870,98740,3.018026e+05,5.060204e+04
50920,2021-06-30,화북동,3749600,1810.431884,18.483512,1.918163e+08,13370520.0,4.846083e+07,5644.0,644.0,1645.0,2255,111933,2.433203e+05,5.097469e+04
